# Data matching

En este Notebook machearemos los elementos HTML que hemos etiquetado en el Notebook anterior. Para ellos haremos uso de de las máscaras de los pantallazos.

Si no se han ejecutado correctamente los Notebook de data extraction y data labeling esta consulta no devolverá resultados.

#### Requirements

In [ ]:
!pip install google-cloud-bigquery
!pip install google-cloud-storage
!pip install Pillow

In [ ]:

from PIL import Image
from io import BytesIO

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'tfmfabri-2cf8db9433ba.json'

from google.cloud import bigquery
client = bigquery.Client()

query = 'SELECT distinct url,ts,created_date FROM `tfmfabri.tfm.tfm_urls` where matched != true and labeled = true and scraped = true order by created_date desc'
query_job = client.query(query)
results = query_job.result()


count = 0

for row in results:
    url = ''
    ts = ''
    url = row['url']
    ts = str(row['ts'])

    print(url)

    from google.cloud import storage
    client_storae = storage.Client()
    # https://console.cloud.google.com/storage/browser/[bucket-id]/

    bucket = client_storae.get_bucket('tfm_bucket')

    filename = ts+'_mask.png'

    #print(filename)

    blob = bucket.blob(filename)


    mask = blob.download_as_string()
    im = Image.open(BytesIO(mask))

    query = "SELECT * FROM `tfmfabri.tfm.tfm_label_cookies` where url = '"+url+"' "
    query_job = client.query(query)
    results = query_job.result()
    pixel_cookies = ''
    pixel_aceptar = ''
    pixel_cerrar = ''
    for row in results:
        pixel_cookies = row['cookies']
        pixel_aceptar = row['aceptar']
        pixel_cerrar = row['cerrar']

    rgb_im = im.convert('RGB')


    # Mensaje de cookies
    if pixel_cookies != '':
        split_pixel = pixel_cookies.split(",")

        #print(split_pixel)
        r, g, b = rgb_im.getpixel((int(split_pixel[0]),int(split_pixel[1])))
        r = str(r).zfill(3)
        g = str(g).zfill(3)
        b = str(b).zfill(3)
        rgb_color = "('"+r+"', '"+g+"', '"+b+"')"
        print("cookies")
        print(rgb_color)
        #query = "SELECT * FROM `tfmfabri.tfm.tfm_url_df` where url = '"+url+"' and rbg_color = \""+rgb_color+"\""
        query = "UPDATE `tfmfabri.tfm.tfm_url_df` set label = 'mensaje_cookies',labeled = true where url = '"+url+"' and rbg_color = \""+rgb_color+"\""
        query_job = client.query(query)
        results = query_job.result()

            
            
    # Botón aceptar cookies
    if pixel_aceptar != '':
        split_pixel = pixel_aceptar.split(",")

        #print(split_pixel)
        r, g, b = rgb_im.getpixel((int(split_pixel[0]),int(split_pixel[1])))
        r = str(r).zfill(3)
        g = str(g).zfill(3)
        b = str(b).zfill(3)
        rgb_color = "('"+r+"', '"+g+"', '"+b+"')"
        print("aceptar")

        print(rgb_color)
        #query = "SELECT * FROM `tfmfabri.tfm.tfm_url_df` where url = '"+url+"' and rbg_color = \""+rgb_color+"\""
        query = "UPDATE `tfmfabri.tfm.tfm_url_df` set label = 'aceptar_cookies',labeled = true where url = '"+url+"' and rbg_color = \""+rgb_color+"\""
        query_job = client.query(query)
        results = query_job.result()

        
    # Botón cerrar cookies
    if pixel_cerrar != '':
        split_pixel = pixel_cerrar.split(",")

        #print(split_pixel)
        r, g, b = rgb_im.getpixel((int(split_pixel[0]),int(split_pixel[1])))
        r = str(r).zfill(3)
        g = str(g).zfill(3)
        b = str(b).zfill(3)
        rgb_color = "('"+r+"', '"+g+"', '"+b+"')"
        print("cerrar")        
        print(rgb_color)
        #query = "SELECT * FROM `tfmfabri.tfm.tfm_url_df` where url = '"+url+"' and rbg_color = \""+rgb_color+"\""
        query = "UPDATE `tfmfabri.tfm.tfm_url_df` set label = 'cerrar_cookies',labeled = true where url = '"+url+"' and rbg_color = \""+rgb_color+"\""
        query_job = client.query(query)
        results = query_job.result()            
            
            
            
    query = "UPDATE `tfmfabri.tfm.tfm_urls` set matched = true where url = '"+url+"'"
    query_job = client.query(query)
    results = query_job.result()
    print("count: "+str(count))
    count += 1

    
